<div style="display:flex;">
<img alt="New Relic" style="display:block;height:64px" src="https://raw.githubusercontent.com/dynovaio/open-source-office/refs/heads/master/assets/brands/new_relic.png" />
<img alt="SoftButterfly" style="display:block;height:64px;margin-left:auto" src="https://raw.githubusercontent.com/dynovaio/open-source-office/refs/heads/master/assets/dynova/dynova-logo-195x64.png" />
</div>

# New Relic Playground 4: Service Levels

## Imports

Python Imports

In [ ]:
import os

Third-party libraries

In [ ]:
import dotenv
from newrelic_sb_sdk.client import NewRelicGqlClient
from newrelic_sb_sdk.graphql import nerdgraph
from newrelic_sb_sdk.graphql.objects import RootMutationType, RootQueryType
from newrelic_sb_sdk.utils.query import NULL_CURSOR
from newrelic_sb_sdk.utils.response import get_response_data, print_response
from sgqlc.operation import Operation

## Client setup

To setup the client, first we need to open load the credentials from environment variables.

In [ ]:
env_file = "../.env"

dotenv.load_dotenv(env_file)

NEW_RELIC_USER_KEY = os.environ.get("NEW_RELIC_USER_KEY", None)

if NEW_RELIC_USER_KEY is None:
    raise ValueError("Environment variable NEW_RELIC_USER_KEY is not set.")

NEW_RELIC_ACCOUNT_ID = os.environ.get("NEW_RELIC_ACCOUNT_ID", None)

if NEW_RELIC_ACCOUNT_ID is None:
    raise ValueError("Environment variable NEW_RELIC_ACCOUNT_ID is not set.")

With environment varaibles loaded, we can proceed to instantiate the client

In [ ]:
newrelic = NewRelicGqlClient(new_relic_user_key=NEW_RELIC_USER_KEY)

## Client testing

In order to use an test the client we need configure `query_type` and `mutation_type` for the `nerdgraph` schema

In [ ]:
nerdgraph.query_type = RootQueryType
nerdgraph.mutation_type = RootMutationType

For testing we will use a simple query in GraphQL to get the atttributes from our user

```gql
query {
  actor {
    user {
      email
      id
      name
    }
  }
}
```

This query will be build from the `nerdgraph` schema.

In [ ]:
# Declare an operation from the `query_type` in `nerdgraph`
operation = Operation(nerdgraph.query_type)

# Get the fields `id`, `email`, `name` from the `user` entity inside `actor`
operation.actor.user.__fields__("id", "email", "name")

This operation can be transformed into a GraphQL query

In [ ]:
query = operation.__to_graphql__()
print(query)

And this query is the one we send to be executed by our client.

In [ ]:
response = newrelic.execute(query)

The response data obtained is

In [ ]:
print_response(response)

We can also use the raw query directly written by hand. For this is recomendable to use the `build_query` method in order to get a clean query.

In [ ]:
query = newrelic.build_query(
    """
        {
            actor {
                user {
                    email
                    name
                    id
                }
            }
        }
    """
)

And execute directly, as in the previous execution, with the same result

In [ ]:
response = newrelic.execute(query)
print_response(response)

## Playground area

### Serivices Levels management tasks with Nerdgraph

### Example 01: List all available service levels

Using a raw NRQL query we can list all available service levels with this

In [ ]:
template = """
{
    actor {
        entitySearch(query: "type IN ('SERVICE_LEVEL')") {
            count
            query
            results {
                nextCursor
                entities {
                    accountId
                    alertSeverity
                    domain
                    entityType
                    firstIndexedAt
                    guid
                    indexedAt
                    lastReportingChangeAt
                    name
                    permalink
                    reporting
                    type
                }
            }
        }
    }
}
"""

params = {
    "cursor": NULL_CURSOR,
}

query = newrelic.build_query(
    template,
    params=params,
)

In [ ]:
response = newrelic.execute(query)

In [ ]:
response.json()

In [ ]:
service_level_count = get_response_data(
    response, key_path="entitySearch:count", action="actor"
)
service_level_count

In [ ]:
service_level_list = get_response_data(
    response, key_path="entitySearch:results:entities", action="actor"
)
len(service_level_list)

In [ ]:
service_level_list[0]

In [ ]:
next_cursor = get_response_data(
    response, key_path="entitySearch:results:nextCursor", action="actor"
)
next_cursor

We can also use the Operation and nerdgraph object in the following way

In [ ]:
# Declare an operation from the `query_type` in `nerdgraph`
operation = Operation(nerdgraph.query_type)
entity_search_op = operation.actor.entity_search(query="type IN ('SERVICE_LEVEL')")
entity_search_op.__fields__("count", "query")
entity_search_op.results.next_cursor()
entity_search_op.results.entities.__fields__(
    "account_id",
    "alert_severity",
    "domain",
    "entity_type",
    "first_indexed_at",
    "guid",
    "indexed_at",
    "last_reporting_change_at",
    "name",
    "permalink",
    "reporting",
    "type",
)

operation

In [ ]:
response = newrelic.execute(operation.__to_graphql__())

In [ ]:
response.json()

In [ ]:
data = operation + response.json()

In [ ]:
data.actor.entity_search["count"]

In [ ]:
len(data.actor.entity_search.results.entities)

### Example 02: Get a single service level

For geeting a conditions from a service level is required know the identification this

In [ ]:
# Getting guid the result using a raw NRQL query
service_level_guid = service_level_list[0]["guid"]

In [ ]:
# Getting guid the result using Operation and nerdgraph
service_level_guid = data.actor.entity_search.results.entities[0]["guid"]

Using a raw NRQL query we can get a single service level with this

In [ ]:
template = """
{
    actor {
        entity(guid: "%(service_level_guid)s") {
            accountId
            alertSeverity
            domain
            entityType
            firstIndexedAt
            guid
            indexedAt
            lastReportingChangeAt
            name
            permalink
            reporting
            type
        }
    }
}
"""

params = {
    "service_level_guid": service_level_guid,
}

query = newrelic.build_query(
    template,
    params=params,
)

In [ ]:
response = newrelic.execute(query)

In [ ]:
serivce_level = get_response_data(response, key_path="entity", action="actor")
serivce_level

We can also use the Operation and nerdgraph object in the following way

In [ ]:
# Declare an operation from the `query_type` in `nerdgraph`
operation = Operation(nerdgraph.query_type)
(
    operation.actor.entity(guid=service_level_guid).__fields__(
        "account_id",
        "alert_severity",
        "domain",
        "entity_type",
        "first_indexed_at",
        "guid",
        "indexed_at",
        "last_reporting_change_at",
        "name",
        "permalink",
        "reporting",
        "type",
    )
)

operation

In [ ]:
response = newrelic.execute(operation.__to_graphql__())

In [ ]:
data = operation + response.json()

In [ ]:
data.actor.entity

### Task 01: Create a service level

In [ ]:
template = """
    write your query here!
"""

params = {
    "account_id": NEW_RELIC_ACCOUNT_ID,
}

query = newrelic.build_query(
    template,
    params=params,
)

In [ ]:
response = newrelic.execute(query)

In [ ]:
created_service_level = get_response_data(response, action="put your key here")
created_service_level